In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image

In [42]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [43]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [44]:
!pwd

/root/3A/ProjetLong/OrthopteraStridulationClassifier/Notebooks/XgBoost


In [45]:
# Path Yolo results
path_audible_results_YOLO = "../../../DataSet/soundscape_to_5s/Selection soundscapes 1 min split 5s/"
path_audible_csv_YOLO = "../../../DataSet/soundscape_to_5s/CSV soundscapes 1 min split 5s boxes/1min_to_5s_boxes.csv"
df_yolo = pd.read_csv(path_audible_csv_YOLO)

# path for audio files 
path_audible_audio_files = "../../../DataSet/Selection soundscapes 1 min/"
path_audible_csv = "../../../DataSet/CSVs soundscapes 1 min/selected_soundscapes.csv" 
df = pd.read_csv(path_audible_csv)

In [46]:
# spectrogrammes pour chaque audio
# Charger l'audio
filtered_audio = set()
with open(path_audible_csv, newline='') as fichier_csv:
    lecteur_csv = csv.reader(fichier_csv)
    next(lecteur_csv)
    # Parcourir chaque ligne du fichier CSV
    for ligne in lecteur_csv:
        filtered_audio.add(ligne[1])

audio_list = list(filtered_audio)


In [49]:
import librosa

colonnes = ["unique_code", "max_frequency", "min_frequency", "avg_frequency", "x1", "x2", "y1", "y2" "duree", "start_split", "fin_split", "predicted_label"]
new_df = pd.DataFrame(columns= colonnes)
new_df["unique_code"] = df_yolo["unique_code"]
new_df["start_split"] = df_yolo["start_split"]
new_df["end_split"] = df_yolo["end_split"]
new_df["x1"] = df_yolo["x1"]
new_df["x2"] = df_yolo["x2"]
new_df["y1"] = df_yolo["y1"]
new_df["y2"] = df_yolo["y2"]

# On calcule les spectrogrammes de la liste si dessus 
# on parcourt le csv de yolo pour repérer les bounding box, et on recup les infos qu'on veut : freq et durée 
based_sr = 48000

for audio in audio_list : 
    
    wav_path = path_audible_audio_files + audio + ".wav"
    y, sr = librosa.load(wav_path, sr=based_sr)
    #print(len(y))
    # parcourir toutes les bounding box de cette audio 
    df_inter = df_yolo[df_yolo["unique_code"] == audio]

    for _, row in df_inter.iterrows():
        # convertir les bounding box en sec 
        width = row["img_width"]
        x1 = row["x1"]
        x2 = row["x2"]
        start = row["start_split"]
        end = row["end_split"]
        debut = int((start + x1*5/width)*based_sr)
        fin = int((start + x2*5/width)*based_sr)
        y_crop = y[debut:fin]
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y_crop)), ref=np.max)
        amplitude_spectrale = np.abs(librosa.stft(y_crop))

        # Prendre les infos necessaires dans cette boundingbox 
        freq = librosa.fft_frequencies(sr=based_sr, n_fft=amplitude_spectrale.shape[0])
        new_df["min_frequency"] = freq.min()
        new_df["max_frequency"]  = freq.max()
        new_df["avg_frequency"]  = freq.mean()
        new_df["duree"] = end - start

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


In [50]:
# Préparation des données 
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

path_global_5s_train = "../../../DataSet/CSVs_morceaux_audio_5s/Audible/train_audible_recording_chunks.csv"
df_train = pd.read_csv(path_global_5s_train)
df_train = df_train[df_train["label_class"] == "Insecta"]

label_encoder = LabelEncoder()
label_encoder.fit(df_train['label_order'])

y_label = label_encoder.transform(df_train['label_order'])

df_train['duree'] = df_train['duree_min'] * 60 + df_train['duree_sec']
df_train = df_train[['max_frequency', 'min_frequency', 'avg_frequency', 'duree']]
df_train['min_frequency'] = df_train['min_frequency'].astype(int)
df_train['max_frequency'] = df_train['max_frequency'].astype(int)

# Gestion des valeurs manquantes
#df.fillna(df.mean(), inplace=True)
df_train = df_train.dropna()

# Gestion des variables catégorielles
df_train = pd.get_dummies(df_train)

X_train = df_train

#################### Préparer X_test avec les données ci-dessus 
#y_yolo_label = label_encoder.transform(new_df['label']) # A changer

new_dftest = new_df[['max_frequency', 'min_frequency', 'avg_frequency', 'duree']]
new_dftest['min_frequency'] = new_dftest['min_frequency'].astype(int)
new_dftest['max_frequency'] = new_dftest['max_frequency'].astype(int)

# Gestion des valeurs manquantes
#df.fillna(df.mean(), inplace=True)
new_dftest = new_dftest.dropna()

# Gestion des variables catégorielles
new_dftest = pd.get_dummies(new_dftest)

X_test = new_dftest

/tmp/ipykernel_10833/773233697.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dftest['min_frequency'] = new_dftest['min_frequency'].astype(int)
/tmp/ipykernel_10833/773233697.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dftest['max_frequency'] = new_dftest['max_frequency'].astype(int)


In [51]:
# XGBoost
# Initialiser et entraîner le modèle XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=2)

# Calculer les poids des classes
# 'balanced' ajuste automatiquement les poids inversément proportionnels aux fréquences des classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_label), y=y_label)

# Créer un dictionnaire des poids pour faciliter l'application des poids aux échantillons
weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Appliquer les poids aux instances de formation
sample_weight = np.array([weights_dict[y] for y in y_label])

# Entraînement du modèle avec les poids des instances
xgb_model.fit(X_train, y_label, sample_weight=sample_weight)

predictions_label_xgb = xgb_model.predict(X_test)

# Évaluation LightGBM
#print("Évaluation Yolo :")
#print("Accuracy sur l'ensemble de validation:", accuracy_score(y_yolo_label, predictions_label))
#print(classification_report(y_yolo_label, predictions_label))

# matrice de confusion
#cm = confusion_matrix(y_yolo_label, predictions_label)
#y_test_inversed = label_encoder.inverse_transform(y_yolo_label)

#unique_labels = np.unique(y_test_inversed)

# plt.figure(figsize=(30,30))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=unique_labels, yticklabels=unique_labels)
# plt.title('Matrice de confusion - Labels')
# plt.ylabel('Valeur réelle')
# plt.xlabel('Valeur prédite')
# plt.show()

In [52]:
print(predictions_label_xgb)

[2 2 2 ... 2 2 2]


In [53]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")

rf_classifier.fit(X_train, y_label)

predictions_label_randomf = rf_classifier.predict(X_test)

# # Évaluation LightGBM
# print("Évaluation LightGBM :")
# print("Accuracy sur l'ensemble de validation:", accuracy_score(y_label, predictions_label))
# print(classification_report(y_label, predictions_label))

# # matrice de confusion
# cm = confusion_matrix(y_label, predictions_label)
# y_test_inversed = label_encoder.inverse_transform(y_label)

# unique_labels = np.unique(y_test_inversed)

# plt.figure(figsize=(30,30))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=unique_labels, yticklabels=unique_labels)
# plt.title('Matrice de confusion - Labels')
# plt.ylabel('Valeur réelle')
# plt.xlabel('Valeur prédite')
# plt.show()

In [54]:
print(predictions_label_randomf)

[0 0 0 ... 0 0 0]


In [59]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm

df_truth = df = pd.read_csv(path_audible_csv)
# TODO : génerer les spectrogrammes 
soundscape_image_path = "../../../DataSet/Spectro_1min/"

def image_to_compare(code_unique, save_path, df_yolo) : 

    df_yolo_filtered = df_yolo[df_yolo["unique_code"] == code_unique]
    df_truth_filtered = df_truth[(df_truth["code_unique"] == code_unique) & (df_truth["label_class"] == "Insecta")]

    chunk_duration = (df_truth_filtered["duree_min"] * 60 + df_truth_filtered["duree_sec"]).iloc[0]

    image = Image.open(f'{soundscape_image_path}{code_unique}.png')
    img_width, img_height = image.width, image.height

    # On fait le truth
    image = Image.open(f'{soundscape_image_path}{code_unique}.png')
    img_width, img_height = image.width, image.height

    plt.figure(figsize=(50, 10))

    for _, row in df_yolo_filtered.iterrows():
        label = row["predicted_label"]
        x1 = row["x1"]
        x2 = row["x2"]
        y1 = row["y1"]
        y2 = row["y2"]
        start_split = row["start_split"] * (img_width // chunk_duration)
        end_split = row["end_split"] * (img_width // chunk_duration)

        t1_pixel = x1 + start_split
        f1_pixel = y1
        f2_pixel = y2
        t2_pixel = x2 + start_split

        rect = patches.Rectangle((t1_pixel, f1_pixel), t2_pixel - t1_pixel, f2_pixel - f1_pixel, linewidth=2, edgecolor='yellow', facecolor='none')
        plt.gca().add_patch(rect)

        plt.text(t1_pixel, f1_pixel + 0.01, label, verticalalignment='bottom', horizontalalignment='left',
            color='black',  # Couleur du texte
            bbox=dict(facecolor='yellow', edgecolor='none', pad=2.0))

    plt.imshow(image)
    plt.title(f'{code_unique} predicted')
    plt.tight_layout()
    plt.savefig(f'{save_path}/{code_unique}_predicted.png')

    plt.figure(figsize=(50, 10))

    for _, row in df_truth_filtered.iterrows():
        chunk_initial_time = 0.0
        chunk_final_time = 60.0
        annotation_initial_time = row["annotation_initial_time"]
        annotation_final_time = row["annotation_final_time"]
        min_frequency = row["min_frequency"]
        max_frequency = row["max_frequency"]
        label = row["label"]

        # Create the BB
        t1 = row["annotation_initial_time"]
        t2 = row["annotation_final_time"]
        f1 = int(row['min_frequency'])
        f2 = int(row['max_frequency'])
        sr = 48000 #row['sampling_rate']
        #sr = row['sampling_rate']

        t1_pixel = int(t1 * img_width/chunk_duration)
        t2_pixel = int(t2 * img_width/chunk_duration)

        # for the frequency, the spectrogram is flipped
        f1_pixel = img_height - int(f1 * img_height/(sr/2))
        f2_pixel = img_height - int(f2 * img_height/(sr/2))

        rect = patches.Rectangle((t1_pixel, f1_pixel), t2_pixel - t1_pixel, f2_pixel - f1_pixel, linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

        plt.text(t1_pixel, f2_pixel + 0.01, label, verticalalignment='bottom', horizontalalignment='left',
            color='white',  # Couleur du texte
            bbox=dict(facecolor='red', edgecolor='none', pad=2.0))

    plt.imshow(image)
    plt.title(f'{code_unique} truth')
    plt.tight_layout()
    plt.savefig(f'{save_path}/{code_unique}_truth.png')
    plt.clf()
    plt.cla()
    
    print(f'{code_unique} fini')

In [60]:
# print les BB 
# Pour randomforest
df_randomf = new_df
df_randomf['predicted_label'] = predictions_label_randomf

code_uniques = df_truth["code_unique"].unique()
save_path = "../../../DataSet/ResultsRand"

for code_unique in tqdm(code_uniques):
    image_to_compare(code_unique, save_path, df_randomf)

  0%|          | 1/243 [00:07<30:04,  7.45s/it]

Orthoptera_cdpotsa09FR_05X2022_806 fini


  1%|          | 2/243 [00:12<24:40,  6.14s/it]

Orthoptera_plmarsa31FR_12V2023_1213 fini


  1%|          | 3/243 [00:17<22:42,  5.68s/it]

...

 40%|███▉      | 96/243 [07:15<10:08,  4.14s/it]

Aves_27Esparr31FR_23IV2019_1300 fini


 40%|███▉      | 97/243 [07:23<12:54,  5.30s/it]

Aves_28StAndre31FR_28V2020_1300 fini


: 

In [1]:
# Pour XGBoost
df_xgb = new_df
df_xgb['predicted_label'] = predictions_label_xgb

code_uniques = df_truth["code_unique"].unique()
save_path = "../../../DataSet/ResultsXGB"

for code_unique in tqdm(code_uniques):
    image_to_compare(code_unique, save_path, df_xgb)

NameError: name 'new_df' is not defined